### 第七章：函数
#### 7.1 可接受任意数量参数的函数
你想构造一个可接受任意数量参数的函数。

In [1]:
def avg(first, *rest):
    return (first + sum(rest) / (1 + len(rest)))

In [2]:
print(avg(1, 2))
print(avg(1, 2, 4, 5, 6))

2.0
4.4


#### 7.3 给函数参数增加元信息
你写好了一个函数，然后想为这个函数的参数增加一些额外的信息，这样的话其他
使用者就能清楚的知道这个函数应该怎么使用。

In [1]:
def add(x:int, y:int) -> int:
    return x + y

In [2]:
help(add)
print(add.__annotations__)

Help on function add in module __main__:

add(x: int, y: int) -> int

{'x': <class 'int'>, 'y': <class 'int'>, 'return': <class 'int'>}


#### 7.10 带额外状态信息的回调函数
你的代码中需要依赖到回调函数的使用 (比如事件处理器、等待后台任务完成后的
回调等)，并且你还需要让回调函数拥有额外的状态值，以便在它的内部使用到。

In [12]:
def apply_async(func, args, *, callback):
    result = func(*args)
    
    callback(result)

In [13]:
def print_result(result):
    print('Got:', result)

def add(x, y):
    return x + y

In [14]:
apply_async(add, (2, 3), callback=print_result)
apply_async(add, ('Hello', 'world'), callback=print_result)

Got: 5
Got: Helloworld


In [26]:
class ResultHandler:

    def __init__(self):
        self.sequence = 0

    def handler(self, result):
        self.sequence += 1
        print('[{}] Got: {}'.format(self.sequence, result))


def make_handler():
    sequence = 0

    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    
    return handler

def make_handler1():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))

In [27]:
r = ResultHandler()
r1 = make_handler()

In [28]:
apply_async(add, (2, 3), callback=r1)
apply_async(add, ('Hello', 'world'), callback=r1)

[1] Got: 5
[2] Got: Helloworld


In [29]:
handler = make_handler1()
next(handler)
apply_async(add, (2, 3), callback=handler.send)
apply_async(add, ('Hello', 'world'), callback=handler.send)

[1] Got: 5
[2] Got: Helloworld


### 第八章：类与对象
#### 8.1 改变对象的字符串显示
你想改变对象实例的打印或显示输出，让它们更具可读性。

In [8]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
    
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)

In [9]:
p = Pair(3, 4)
print(p)
print(repr(p))

(3, 4)
Pair(3, 4)


In [11]:
print('p is {0!r}'.format(p))
print('p is {0}'.format(p))

p is Pair(3, 4)
p is (3, 4)


#### 8.2 自定义字符串的格式化
你想通过 format() 函数和字符串方法使得一个对象能支持自定义的格式化。

In [13]:
_formats = {
'ymd' : '{d.year}-{d.month}-{d.day}',
'mdy' : '{d.month}/{d.day}/{d.year}',
'dmy' : '{d.day}/{d.month}/{d.year}'
}

In [15]:
class Date:
    
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [23]:
d = Date(2012, 12, 21)
print(format(d, 'dmy'))
print('The date is {:dmy}'.format(d))
print('The date is {:mdy}'.format(d))

21/12/2012
The date is 21/12/2012
The date is 12/21/2012


In [25]:
from datetime import date
d = date(2020, 1, 20)

In [28]:
print(format(d))
print(format(d, '%A, %B, %d, %Y'))
print('The end is {:%d %b %Y}. Goodbye'.format(d))

2020-01-20
Monday, January, 20, 2020
The end is 20 Jan 2020. Goodbye


#### 8.3 让对象支持上下文管理协议
你想让你的对象支持上下文管理协议 (with 语句)。

In [34]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    
    def __init__(self, address, family=AF_INET, tp=SOCK_STREAM):
        self.address = address
        self.family = family
        self.tp = tp
        self.sock = None
    
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected!')
        self.sock = socket(self.family, self.tp)
        self.sock.connect(self.address)
        return self.sock
    
    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

In [35]:
from functools import partial

conn = LazyConnection(('www.python.org', 80))

with conn as s:
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
print(resp)

b'HTTP/1.1 301 Moved Permanently\r\nServer: Varnish\r\nRetry-After: 0\r\nLocation: https://www.python.org/index.html\r\nContent-Length: 0\r\nAccept-Ranges: bytes\r\nDate: Mon, 20 Jan 2020 06:54:48 GMT\r\nVia: 1.1 varnish\r\nConnection: close\r\nX-Served-By: cache-tyo19930-TYO\r\nX-Cache: HIT\r\nX-Cache-Hits: 0\r\nX-Timer: S1579503289.543130,VS0,VE0\r\nStrict-Transport-Security: max-age=63072000; includeSubDomains\r\n\r\n'


#### 8.4 创建大量对象时节省内存方法
你的程序要创建大量 (可能上百万) 的对象，导致占用很大的内存。

In [36]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

#### 8.5 在类中封装属性名
你想封装类的实例上面的“私有”数据，但是 Python 语言并没有访问控制。

In [37]:
class A:
    def __init__(self):
        self._internal = 0
        self.public = 1
        
    def public_method(self):
        pass
    
    def _internal_method(self):
        pass

In [38]:
class B:
    def __init__(self):
        self.__private = 0
    
    def __private_method(self):
        pass
    
    def public_method(self):
        pass
        self.__private_method()

In [41]:
class C(B):
    def __init__(self):
        super().__init__()
        self.__private = 1
    
    def __private_method(self):
        pass

In [48]:
c = C()
print(dir(c))

['_B__private', '_B__private_method', '_C__private', '_C__private_method', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'public_method']


#### 8.6 创建可管理的属性
你想给某个实例 attribute 增加除访问与修改之外的其他处理逻辑，比如类型检查
或合法性验证。

In [56]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
    
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")

In [66]:
a = Person('Guido')

In [67]:
print(a.first_name)
a.first_name = 'ling'
del a.first_name

Guido


AttributeError: Can't delete attribute

#### 8.7 调用父类方法
你想在子类中调用父类的某个已经被覆盖的方法。

In [6]:
class A:
    def __init__(self):
        self.x = 0

class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

In [8]:
b = B()
print(b.x, b.y)

0 1


In [10]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')

class B(Base):
    def __init__(self):
        Base.__init__(self)
        print('B.__init__')

class C(A, B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
        print('C.__init__')

In [11]:
c = C()

Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


In [12]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')

class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')

class C(A, B):
    def __init__(self):
        super().__init__()
        print('C.__init__')

In [13]:
c = C()

Base.__init__
B.__init__
A.__init__
C.__init__


In [14]:
print(C.__mro__)

(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>)


#### 8.8 子类中扩展 property
在子类中，你想要扩展定义在父类中的 property 的功能。

In [16]:
class Person:
    def __init__(self, name):
        self.name = name
    # Getter function
    @property
    def name(self):
        return self._name
    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value
    # Deleter function
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

In [17]:
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name

    @name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)

In [18]:
s = SubPerson('Guido')
print(s.name)
s.name = 'Ling'

Setting name to Guido
Getting name
Guido
Setting name to Ling


In [22]:
#扩展 property 的某一个方法
class SubPerson1(Person):
    @Person.name.getter
    def name(self):
        print('Getting name')
        return super().name
# 只想修改 setter 方法
class SubPerson2(Person):
    @Person.name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson2, SubPerson2).name.__set__(self, value)

#### 8.9 创建新的类或实例属性
你想创建一个新的拥有一些额外功能的实例属性类型，比如类型检查。

In [23]:
class Integer:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected a int')
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]

In [24]:
class Point:
    x = Integer('x')
    y = Integer('y')
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [25]:
p = Point(2, 3)
print(p.x)
p.y = 5
p.x = 2.3

2


TypeError: Expected a int

In [26]:
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected '+ str(self.expected_type))
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        del instance.__dict__[self.name]

In [27]:
def typeassert(**kwargs):
    def decorate(cls):
        for name, expected_type in kwargs.items():
            setattr(cls, name, Typed(name, expected_type))
        return cls
    return decorate

In [28]:
@typeassert(name=str, shares=int, price=float)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

#### 8.10 使用延迟计算属性
你想将一个只读属性定义成一个 property，并且只在访问的时候才会计算结果。但
是一旦被访问后，你希望结果值被缓存起来，不用每次都去计算。

In [29]:
class lazyproperty:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

In [30]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

In [32]:
c = Circle(4.0)
print(c.radius)
print(c.area)
print(c.area)

4.0
Computing area
50.26548245743669
50.26548245743669


#### 8.11 简化数据结构的初始化
你写了很多仅仅用作数据结构的类，不想写太多烦人的 \_\_init\_\_() 函数
#### 8.12 定义接口或者抽象基类
你想定义一个接口或抽象类，并且通过执行类型检查来确保子类实现了某些特定
的方法

In [1]:
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass
    
    @abstractmethod
    def write(self, data):
        pass

In [2]:
a = IStream()

TypeError: Can't instantiate abstract class IStream with abstract methods read, write

In [3]:
class SocketStream(IStream):
    def read(self, maxbytes=-1):
        pass
    def write(self, data):
        pass

In [5]:
a = SocketStream()
print(isinstance(a, IStream))

True


In [9]:
import io

IStream.register(io.IOBase)
f = open('foo.txt', 'w')
print(isinstance(f, IStream))
f.close()

True


#### 8.13 实现数据模型的类型约束
你想定义某些在属性赋值上面有限制的数据结构。
#### 8.14 实现自定义容器
你想实现一个自定义的类来模拟内置的容器类功能，比如列表和字典。但是你不确
定到底要实现哪些方法。
#### 8.15 属性的代理访问
你想将某个实例的属性访问代理到内部另一个实例中去，目的可能是作为继承的
一个替代方法或者实现代理模式。